# Engine: openmdao

In [ ]:
from modeling.compute import Var, ureg
from engine.gen1mdao.openmdao import coupled_run, buildidpvars, get_outputs
from modeling.gen1.resultscheck import print_out
from modeling.gen1.compute import eqvar
from trash.inputresolver import eqvars, default_out, default_in, mdf_order
import openmdao.api as om

In [2]:
# From outside
E_service = Var('E_service', 20, unit='W*hr')
# From inside
mu_batt = Var('mu_batt', 30, unit='W*hr/kg')
DOD = Var('DOD', 0.4)
eta_trans = Var('eta_trans', 0.85)
nu_batt = Var('nu_batt', 450, unit='kW*hr/(m**3)')
N = Var('N', 1)
C, eq1 = eqvar('C', E_service/(DOD*N*eta_trans), unit='kW*hr')
m_batt_zero = Var('m_batt_zero', 5, 'kg')
m_batt, eq2 = eqvar('m_batt', C/mu_batt + m_batt_zero, 'kg')
Vol_batt, eq3 = eqvar('Vol_batt', C/nu_batt, 'm**3')

In [3]:
eqs={idx:eq for idx, eq in enumerate([eq1, eq2, eq3])}
ins = default_in(eqs)
eqv = eqvars(eqs)
dout = default_out(eqs)
order = mdf_order(eqv, dout)
order

[0, 1, 2]

In [4]:
prob = om.Problem()
model = prob.model
counter = coupled_run(eqs, order, (), model, model, 0)
buildidpvars(ins, model)
prob.setup()
prob.run_model()

In [5]:
prob.set_val('N', 1)
prob.run_model()

In [6]:
out = get_outputs(eqs, model)
out

{m_batt_zero: 5.0,
 m_batt: 6.96078431372549,
 nu_batt: 450.0,
 C: 0.0588235294117647,
 N: 1.0,
 mu_batt: 30.0,
 Vol_batt: 0.000130718954248366,
 eta_trans: 0.85,
 DOD: 0.4,
 E_service: 20.0}

In [7]:
20.0*E_service.varunit

20.0 <Unit('hour * watt')>

In [8]:
print_out(out)

{m_batt_zero: '5.00 kg',
 m_batt: '6.96 kg',
 nu_batt: '450.00 h * kW / m ** 3',
 C: '58.82 W * h',
 N: '1.00',
 mu_batt: '30.00 h * mW / g',
 Vol_batt: '130718.95 mm ** 3',
 eta_trans: '0.85',
 DOD: '0.40',
 E_service: '20.00 W * h'}